# Convolutional Neural Networks
---


## ImageNet


![ImageNet](Images/imagenet.jpg)


## What happens if we use this for images? (hint: dimensions)
![Deep](Images/deep.png)


## Neurological Origins


![Cat](Images/cat.jpg)


Hubel and Wiesel. Receptive fields of single neurons in the cat's striate cortex. 1959


* Developed by Yann LeCun (1989)
* Goal was to build automatic mail-sorting machines
* First "real-world" application of Neural Networks


![title](Images/cnn.jpg)  
* Weights are shared across space
* Provides basis for translational invariance

## Convolution


![Convolution](Images/convolution.png)


![Algorithm](Images/alg.png)


![Time Complexity](Images/time.png)


The run-time complexity of the convolutional operation is described by the above equation. Where $F_{n}$ is the number of filters, $H$ is the height of the input image, $W$ is the width of the image, $f_{h}$ is the height of the filter, $f_{w}$ is the width of the filter, and $f_{d}$ is the depth of the filter. It is important to note that the equation only describes the forward pass in a convolutional layer. **Backpropogation through the network is roughly twice as costly as the forward pass. This is because we must compute the loss with respect to the weights and the loss with respect to the inputs.** 




![Matrix](Images/conv_matrix.png)


## Rectified Linear Unit (ReLU)


![ReLU](Images/relu.png)


* Very inexpensive
* Avoid vanishing gradient
* Differentiable at all points except 0
* Can kill neurons (Leaky ReLU)


![Leaky ReLU](Images/leakyrelu.png)


## Pooling



![Pooling](Images/pooling.png)
* Gives small amount of translational invariance at each level
    * Location information of most active neuron is thrown away
* Reduces number of inputs to next layer


## Fully Connected


In [ ]:
# forward-pass of a 3-layer neural network:
f = lambda x: 1.0/(1.0 + np.exp(-x)) # activation function (use sigmoid)
x = np.random.randn(3, 1) # random input vector of three numbers (3x1)
h1 = f(np.dot(W1, x) + b1) # calculate first hidden layer activations (4x1)
h2 = f(np.dot(W2, h1) + b2) # calculate second hidden layer activations (4x1)
out = np.dot(W3, h2) + b3 # output neuron (1x1)

## Softmax Output


$$y_{i} = \dfrac{e^{z_{i}}}{\sum_{j=1}^{N}e^{z_{j}}}$$
* Output sums to one
* Represent probability distribution across discrete mutually exclusive alternatives


In [ ]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

## [Softmax Derivative](https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/)


$$\frac{\partial y_{i}}{\partial z_{i}} = y_{i}(1-y_{i})$$

In [ ]:
def softmax_derivative(y):
    return y * (1 - y)

## Cross-entropy Cost Function


$$C = - \sum_{j} t_{j} \log y_{j}$$

* Very large gradient when the target value is 1 and the output is 0

$$\frac{\partial C}{\partial z_{i}} = \sum_{j} \frac{\partial C}{\partial y_{i}} \frac{\partial y_{i}}{\partial z_{i}} = y_{i} - t_{i} $$


In [3]:
def cost(y,t):
    # y: predicted, t: target
    return y - t

## Dropout


![Dropout](Images/dropout.png)

* Combat overfitting
* Randomly drop connections during training
* Regularization


## Understanding CNN's


#### [Weight Visualization](https://www.youtube.com/watch?v=AgkfIQ4IGaM)


![Conv1](Images/conv1.png)


![Conv3](Images/conv3.png)

![Conv5](Images/conv5.png)

#### [Class Activation Mapping](http://cnnlocalization.csail.mit.edu/)


![Code](Images/code.png)


## [TensorFlow](https://www.tensorflow.org/)

* Open source software library for numerical computation using data flow graphs
* Nodes in the graph represent mathematical operations, while the graph edges represent the multidimensional data arrays (tensors)
* Allows you to deploy computation to one or more CPUs or GPUs in a desktop, server, or mobile device with a single API
* Originally developed by researchers and engineers working on the Google Brain Team


## [Keras](https://keras.io/)

* High level API
* Able to use different libraries as backend
    * TensorFlow, Theano, CNTK (Microsoft)
    

## Examples

![MNIST](Images/mnist.jpg)


In [4]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [5]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

('x_train shape:', (60000, 28, 28, 1))
(60000, 'train samples')
(10000, 'test samples')


In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 96s - loss: 0.3352 - acc: 0.8978 - val_loss: 0.0764 - val_acc: 0.9757
Epoch 2/12
60000/60000 [==============================] - 95s - loss: 0.1092 - acc: 0.9679 - val_loss: 0.0506 - val_acc: 0.9834
Epoch 3/12
60000/60000 [==============================] - 95s - loss: 0.0809 - acc: 0.9757 - val_loss: 0.0420 - val_acc: 0.9861
Epoch 4/12
60000/60000 [==============================] - 96s - loss: 0.0693 - acc: 0.9803 - val_loss: 0.0396 - val_acc: 0.9868
Epoch 5/12
60000/60000 [==============================] - 95s - loss: 0.0607 - acc: 0.9818 - val_loss: 0.0339 - val_acc: 0.9886
Epoch 6/12
60000/60000 [==============================] - 94s - loss: 0.0535 - acc: 0.9842 - val_loss: 0.0326 - val_acc: 0.9890
Epoch 7/12
60000/60000 [==============================] - 100s - loss: 0.0486 - acc: 0.9854 - val_loss: 0.0306 - val_acc: 0.9892
Epoch 8/12
60000/60000 [=============================

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.027582556404452771)
('Test accuracy:', 0.99080000000000001)


## Transfer Learning


![Transfer Learning](Images/transfer.png)

In [10]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras import backend as K

In [12]:
input_shape = (300,300,3)
input_tensor = Input(shape=input_shape)
base_model = VGG16(input_tensor=input_tensor, include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x)
# output layer with 'num_classes' number of catergories
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

## Installations


Instructions to install CUDA 8.0, CuDNN, TensorFlow, and Keras can be found [here](https://github.com/jordanott/Useful-Things/blob/master/CUDA.md).


## References

[Stanford CS 231n](http://cs231n.github.io/convolutional-networks/)  
[Keras](https://keras.io/)  
[Keras Tutorials](https://github.com/keras-team/keras/tree/master/examples)  
[TensorFlow](https://www.tensorflow.org/)  
[Deep Visualization Toolbox](https://www.youtube.com/watch?v=AgkfIQ4IGaM)  
[Transfer Learning using Keras](https://towardsdatascience.com/transfer-learning-using-keras-d804b2e04ef8)  
[Dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf)  
